In [213]:
!pip3 install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [373]:
import sys
sys.path.append('src/')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch
import ast

# panda view settings
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns

In [285]:
DEVICE = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

print(DEVICE)

cpu


In [215]:
df = pd.read_csv("./data/toxicbias_train.csv")
df_bias = df[df['bias'] == 'bias']
df_neutral = df[df['bias'] == 'neutral']


## Split up dataframe by category

In [216]:
# store unique categories
unique_categories = set()

# Iterate through each entry in the 'category' column
for categories in df['category'].str.split(','):
    for category in categories:
        stripped_category = category.strip()
        if stripped_category and stripped_category.lower() != 'none':
            unique_categories.add(stripped_category)

unique_categories_list = sorted(list(unique_categories))

In [217]:
categorical_dfs = {category: pd.DataFrame(columns=df.columns) for category in unique_categories_list}

# Split categories and add rows to the corresponding dataframes in dictionary
def split_categories_and_add_rows(row):
    categories = row['category'].split(',')
    for category in categories:
        category = category.strip()  # Remove leading/trailing spaces
        if category in categorical_dfs:
            categorical_dfs[category] = categorical_dfs[category].append(row, ignore_index=True)

df_bias.apply(split_categories_and_add_rows, axis=1)

# Print the shape of each category dataframe
for category, category_df in categorical_dfs.items():
    print(f"Dataframe for {category}:")
    print(f"Shape: {category_df.shape}")
    print("\n")

Dataframe for gender:
Shape: (293, 6)


Dataframe for lgbtq:
Shape: (453, 6)


Dataframe for political:
Shape: (172, 6)


Dataframe for race:
Shape: (1768, 6)


Dataframe for religion:
Shape: (1257, 6)




## Split training and test data

In [218]:
training_data = {}
test_data = {}

# Perform the 80-20 split for each category dataframe
for category, df in categorical_dfs.items():
    train_df, test_df = train_test_split(df, test_size=0.8, random_state=42)
    
    training_data[category] = train_df
    test_data[category] = test_df

In [289]:
for category, train_df in training_data.items():
    print('Training data')
    print(f"Dataframe for {category}:")
    print(f"Shape: {train_df.shape}")
    print("\n")

Training data
Dataframe for gender:
Shape: (58, 6)


Training data
Dataframe for lgbtq:
Shape: (90, 6)


Training data
Dataframe for political:
Shape: (34, 6)


Training data
Dataframe for race:
Shape: (353, 6)


Training data
Dataframe for religion:
Shape: (251, 6)




In [288]:
for category, test_df in test_data.items():
    print('Test data')
    print(f"Dataframe for {category}:")
    print(f"Shape: {test_df.shape}")
    print("\n")

Training data
Dataframe for gender:
Shape: (470, 6)


Training data
Dataframe for lgbtq:
Shape: (726, 6)


Training data
Dataframe for political:
Shape: (276, 6)


Training data
Dataframe for race:
Shape: (2282, 6)


Training data
Dataframe for religion:
Shape: (1873, 6)




## Premises

In [220]:
hypotheses = {
  'gender' : [
    'A person belonging to a gender lacks intelligence.',
    'A person belonging to a gender is a criminal.',
    'A person belonging to a gender is violent.',
    'Violence is encouraged against a gender.',
    'A person belonging to a gender is abusive.',
    'A person belonging to a gender should be blamed.',
    'Women make decisions based on emotions.',
    'A gender is unable to do some jobs.',
    'A gender is unable to do something as well as another gender.',
    'A gender should not talk.',
    'A person belonging to a gender is compared to an object.',
    'A gender should be blamed.',
    'A person belonging to a gender has an advantage compared to others.',
    'A person belonging to a gender is misogynist.',
    'A person belonging to a gender is disliked.',
  ],
  'lgbtq' : [
    'Lgbtq persons are believed to have a condition.',
    'Lgbtq should not be part of society.',
    'Violence is encouraged against lgbtq persons.',
    'Lgbtq persons are compared to animals.',
    'Lgbtq persons are believed to have an agenda.',
    'Lgbtq persons should be excluded.',
    'Lgbtq persons should stay home.',
    'Lgbtq persons commit crimes.',
    'Lgbtq persons get preferred to other persons.',
    'Lgbtq persons are seen as not being normal.', 
    'Lgbtq persons should not be allowed to marry.', 
    'Lgbtq persons have a disease.',
    'Lgbtq persons are seen as being disgusting.',
    'Lgbtq persons are not tolerated or desired.',
    'Lgbtq persons are treated differently.'

  ],
  'political' : [ 
      'A person belonging to a political group is stupid.',
      'A person belonging to a political group is crazy.',
      'Things are worse due to a political group.',
      'Members of a political group are lgbtq.',
      'Members of a political group are hypocritical.',
      'Members of a political group dislike a gender or lgbtq.',
      'Members of a political group commit crimes.',
      'Members of a political group are violent.',
      'Members of a political group are hateful.',
      'Members of a political group are compared to Nazis.',
      'Members of a political group are racist.',
      'A person belonging to a political group has a disease.',
      'Members of a political group are taking control.',
      'Members of a political group prefer minorities.',
      'Members of a political group hate their country or race.',
  ],
  'race' : [
    'Members of a race commit crimes.',
    'Members of a race are violent.',
    'Members of a race are compared to animals.',
    'The use of violence is encouraged or desired against a race.',
    'Members of a race should remain in their respective countries.',
    'The integration of a race should be avoided in a country or society.',
    'Members of a race are a threat to others.',
    'A race is inferior when compared to another race.',
    'A race is preferred to another.',
    'Members of a race are stupid.',
    'There are too many members of a race.',
    'Members of a race should be excluded.',
    'Members of a race should go back to their homes.',
    'Members of a race are refused.',
    'Members of a race are racist.'
  ],
  'religion' : [
    'The use of violence is encouraged or desired against a religion.',
    'A religion should be discriminated against.',
    'Members of a religion should be excluded and marginalized.',
    'Members of a religion should remain in their respective countries.',
    'Members of a religion are compared to animals.',
    'A religion or members of a religion are compared to devils or demons.',
    'The integration of a religion should be avoided in a country or society.',
    'A religion or members of a religion are stupid.',
    'A religion is inferior when compared to another religion.',
    'A religion is preferred to another.'
    'Religious practices of a particular faith should be restricted or banned in public spaces.',
    'Religious attire or symbols should be prohibited.',
    'A religion promotes violence or intolerance.',
    'Members of a religion are a threat.',
    'Members of a religion are not trustworthy or loyal.'
  ],
}

## Add neutral cases

In [221]:
def add_rows_to_dataframes(dataframes_dict, new_dataframe):

    for key, df in dataframes_dict.items():
        # Calculate the maximum number of rows to add
        max_rows_to_add = min(len(new_dataframe), len(df))

        # Choose random rows from the new dataframe
        rows_to_add = new_dataframe.sample(n=max_rows_to_add, replace=True)

        rows_to_add = new_dataframe.iloc[:max_rows_to_add]
        df = pd.concat([df, rows_to_add], ignore_index=True)
        dataframes_dict[key] = df


In [223]:
add_rows_to_dataframes(test_data, df_neutral)

## Bart model

In [286]:
def convert_probabilities_batched(probabilities, label_mapping):
    probabilities_list = probabilities.tolist()  # Convert the tensor to a list
    return [{name: round(pred * 100, 1) for pred, name in zip(preds, label_mapping)} for preds in probabilities_list]

In [287]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def bart_nli_batched_hypotheses(premises, hypotheses):
    model_name = 'facebook/bart-large-mnli'
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model.to(DEVICE) 

    num_premises = len(premises)
    num_hypotheses = len(hypotheses)
    results = {}

    with torch.no_grad():
        for i in range(num_premises):
            labels = []

            # Tokenize the input pair
            inputs = tokenizer([premises[i]] * num_hypotheses, hypotheses, return_tensors='pt', padding=True, truncation=True)

            # Run the input through the model
            logits = model(**inputs.to(DEVICE)).logits

            # Get probabilities and labels for the input
            probabilities = torch.softmax(logits, dim=1)
            label_mapping = ['contradiction', 'neutral', 'entailment']
            labels = [label_mapping[p.argmax().item()] for p in probabilities]

            results[premises[i]] = (labels, convert_probabilities_batched(probabilities, label_mapping))
    return results


## Run data through model, add labels and probabilities

In [267]:
def run_bart_model_add_labels(category):
    # Create a copy of the input dataframe
    df_copy = test_data[category].copy()

    # Run BART NLI on the dataframes for the specified category
    results = bart_nli_batched_hypotheses(list(df_copy['comment_text']), hypotheses[category])

    # Process the results separately and add them to the dataframe
    labels_column = []
    probabilities_column = []

    for premise in df_copy['comment_text']:
        if premise in results:
            labels, probabilities = results[premise]
        else:
            labels, probabilities = None, None
        labels_column.append(labels)
        probabilities_column.append(probabilities)

    df_copy['labels'] = labels_column
    df_copy['probabilities'] = probabilities_column

    return df_copy

In [ ]:
df_results = run_bart_model_add_labels('political')

## Calculate f1 scores

In [387]:
def determine_label_accuracy(df, total_checked=len(hypotheses['race'])):
    def is_correct_bias(row, n_labels):
        labels = ast.literal_eval(row['labels'])
        return sum(label == 'entailment' for label in labels) >= n_labels

    def is_correct_neutral(row):
        labels = ast.literal_eval(row['labels'])
        return all(label != 'entailment' for label in labels)

    # Initialize a list to store the boolean results for each row
    label_correctness_list = []

    # Iterate over each row
    for _, row in df.iterrows():
        # Initialize a list to store the results for this row
        row_results = []
        # Iterate from 1 to n (inclusive) to calculate is_label_correct for this row
        for n in range(1, total_checked + 1):
            result = is_correct_bias(row, n) if row['bias'] == 'bias' else is_correct_neutral(row)
            row_results.append(result)
        # Append the row results to the main list
        label_correctness_list.append(row_results)

    # Add the list as the 'is_label_correct' column in the DataFrame
    df['is_label_correct'] = pd.Series(label_correctness_list)

    df['predicted'] = True
    return df


In [344]:
df_results = determine_label_accuracy(df_results)

/var/folders/yy/fwc73zl141q0zj_v307n1l9w0000gn/T/ipykernel_91610/3546598877.py:21: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df['is_label_correct'] = pd.Series(label_correctness_dict)


In [ ]:
# Function to calculate and print F1 score for a given DataFrame
def print_f1_score(df, category_name):
    f1 = f1_score(df['is_label_correct'], df['predicted'])
    print(f"F1 score for {category_name}: {f1}")

In [388]:
# Read the CSV files into DataFrames
df_political_results = pd.read_csv('./results/final/political_results.csv')
df_gender_results = pd.read_csv('./results/final/gender_results.csv')
df_lgbtq_results = pd.read_csv('./results/final/lgbtq_results.csv')
df_race_results = pd.read_csv('./results/final/race_results.csv')
df_religion_results = pd.read_csv('./results/final/religion_results.csv')

df_political_results.drop('is_label_correct', axis=1, inplace=True)
df_gender_results.drop('is_label_correct', axis=1, inplace=True)
df_lgbtq_results.drop('is_label_correct', axis=1, inplace=True)
df_race_results.drop('is_label_correct', axis=1, inplace=True)
df_religion_results.drop('is_label_correct', axis=1, inplace=True)

In [389]:
df_gender_results = determine_label_accuracy(df_gender_results, 15)

In [313]:
# Combine the DataFrames vertically
combined_df = pd.concat([df_political_results, df_gender_results, df_lgbtq_results, df_race_results, df_religion_results], ignore_index=True)

combined_results = './results/final/combined_results.csv'
combined_df.to_csv(combined_results, index=False)

In [321]:
print_f1_score(combined_df, 'Combined')

F1 score for Combined: 0.8153684210526316


In [392]:
df_gender_results.sample(5)

,Unnamed: 0,comment_text,identity_attack_x,bias,target,category,rationale,labels,probabilities,predicted,is_label_correct
455,455,Yessireee bobbereno Mighty fine move. Mighty fine About fricken time. And white folks are jonesing again and again and again. Always jonesing. You cant have what you had for thousands and thousands of years because I cant have it even if I never did. Hello? Get a life,0.257143,neutral,NaN,none,NaN,"['contradiction', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'contradiction', 'contradiction', 'entailment', 'contradiction', 'contradiction', 'entailment', 'neutral', 'entailment', 'contradiction', 'entailment']","[{'contradiction': 90.1, 'neutral': 6.7, 'entailment': 3.2}, {'contradiction': 92.6, 'neutral': 7.1, 'entailment': 0.3}, {'contradiction': 67.4, 'neutral': 17.5, 'entailment': 15.1}, {'contradiction': 38.4, 'neutral': 56.5, 'entailment': 5.0}, {'contradiction': 64.4, 'neutral': 23.0, 'entailment': 12.6}, {'contradiction': 57.8, 'neutral': 29.4, 'entailment': 12.8}, {'contradiction': 49.9, 'neutral': 28.1, 'entailment': 22.0}, {'contradiction': 14.3, 'neutral': 25.9, 'entailment': 59.8}, {'co...",True,"[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]"
323,323,Would not an ultimate racist be a disturbed halfwhite man who longs to be black because his absent father was black and dislikes a majority white country.,0.533333,neutral,NaN,none,NaN,"['contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'entailment', 'contradiction', 'contradiction', 'contradiction', 'entailment']","[{'contradiction': 81.8, 'neutral': 12.2, 'entailment': 6.0}, {'contradiction': 75.1, 'neutral': 14.0, 'entailment': 10.9}, {'contradiction': 52.4, 'neutral': 23.8, 'entailment': 23.8}, {'contradiction': 84.0, 'neutral': 14.7, 'entailment': 1.3}, {'contradiction': 76.0, 'neutral': 14.8, 'entailment': 9.2}, {'contradiction': 62.6, 'neutral': 12.7, 'entailment': 24.6}, {'contradiction': 65.7, 'neutral': 32.4, 'entailment': 1.9}, {'contradiction': 53.6, 'neutral': 29.6, 'entailment': 16.8}, {'c...",True,"[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]"
75,75,Trump is showing a little too much here. He is called a racist bigot and much worse by higher profile people than an ESPN host. Yet where is the outrage towards them? Oooohhhh this just happens to be a black female. Aaaahhhh... OK.,0.500000,bias,"blacks,women","race,gender",prejudice towards black women,"['contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'contradiction', 'contradiction', 'entailment', 'contradiction', 'contradiction', 'contradiction', 'entailment']","[{'contradiction': 83.1, 'neutral': 14.4, 'entailment': 2.5}, {'contradiction': 95.1, 'neutral': 4.6, 'entailment': 0.3}, {'contradiction': 80.8, 'neutral': 10.6, 'entailment': 8.5}, {'contradiction': 57.7, 'neutral': 38.9, 'entailment': 3.4}, {'contradiction': 76.4, 'neutral': 14.6, 'entailment': 9.0}, {'contradiction': 87.3, 'neutral': 8.3, 'entailment': 4.4}, {'contradiction': 14.1, 'neutral': 74.8, 'entailment': 11.1}, {'contradiction': 43.7, 'neutral': 39.7, 'entailment': 16.5}, {'contr...",True,"[True, True, False, False, False, False, False, False, False, False, False, False, False, False, False]"
47,47,Show me any Imam that says homosexuals women and Atheists are equal to Muslim men...and Ill show you an Imam with a Fatwa on him...,0.800000,bias,"muslims,homosexuals,women,atheists","religion,lgbtq,gender","prejudice against muslims,homosexuals,women,atheists","['contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'contradiction', 'entailment', 'contradiction', 'contradiction', 'contradic